# Chapter01-02

## 0. Prepare

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import warnings

In [2]:
#需要的文件
warnings.filterwarnings('ignore')
path1='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/1-CRSP/CRSP MonthlyStock/MonthlyStock.csv'
path2='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/1-CRSP/CRSP DailyStock/DailyStock.csv'
path3='D:/项目文件/项目文件-个人/Fintech\Fintech-Code/资产定价/F-F_Research_Data_Factors_year.csv'
path4='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/2-Compustat/Fundamentals Annual/0e7f1f3cacd9fa77.csv'
path5='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/2-Compustat/Fundamentals Quarter/fcf28ab9962a3038.csv'
path6='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/2-Compustat/Compustat-CRSP Link/7d3ba590b1864458.csv'
path7='D:/项目文件/项目文件-个人/Fintech\Fintech-Code/资产定价/F-F_Research_Data_Factors_monthly.csv'

In [3]:
#获取列数据
def getColumns(path):
    f=open(path)
    df=pd.read_csv(f,nrows=5)
    f.close()
    return list(df.columns)
columns1=getColumns(path1)
columns2=getColumns(path2)

In [5]:
#选取月度数据
MonthlyStock=pd.read_csv(open(path1),usecols=['date','PERMNO','SHRCD','SHROUT','ALTPRC','RET'])    
#筛选美国普通股
MonthlyStock=MonthlyStock[(MonthlyStock['SHRCD']==10) | (MonthlyStock['SHRCD']==11)]
MonthlyStock.head(10)

,PERMNO,date,SHRCD,RET,SHROUT,ALTPRC
1,10000,19860131,10.0,C,3680.0,-4.37500
2,10000,19860228,10.0,-0.257143,3680.0,-3.25000
3,10000,19860331,10.0,0.365385,3680.0,-4.43750
4,10000,19860430,10.0,-0.098592,3793.0,-4.00000
5,10000,19860530,10.0,-0.222656,3793.0,-3.10938
6,10000,19860630,10.0,-0.005025,3793.0,-3.09375
7,10000,19860731,10.0,-0.080808,3793.0,-2.84375
8,10000,19860829,10.0,-0.615385,3793.0,-1.09375
9,10000,19860930,10.0,-0.057143,3793.0,-1.03125
10,10000,19861031,10.0,-0.242424,3843.0,-0.78125


## 1.MktCap

In [6]:
#MktCap------------------------------------------------------------------
MonthlyStock['year']=MonthlyStock['date']//10000
MonthlyStock['month']=(MonthlyStock['date']%10000)//100

data=MonthlyStock[MonthlyStock['month']==12]
data['MktCap']=np.abs(data['SHROUT']*data['ALTPRC'])/1000

MktCap=data[(data['year']>=1988) & (data['year']<=2012)]
MktCap=MktCap.sort_values(['date','PERMNO'])
MktCap=MktCap[['PERMNO','year','MktCap']]
MktCap.head(10)

,PERMNO,year,MktCap
55,10001,1988,6.362250
438,10002,1988,9.840625
766,10003,1988,41.686000
887,10005,1988,0.785250
1665,10007,1988,NaN
1760,10009,1988,10.586250
1940,10010,1988,15.140250
2028,10011,1988,10.448438
2175,10012,1988,4.765406
3104,10016,1988,400.414000


## 2.Size

In [7]:
Size=MktCap.copy()
Size['Size']=np.log(Size['MktCap'])
Size.head(10)

,PERMNO,year,MktCap,Size
55,10001,1988,6.362250,1.850382
438,10002,1988,9.840625,2.286519
766,10003,1988,41.686000,3.730165
887,10005,1988,0.785250,-0.241753
1665,10007,1988,NaN,NaN
1760,10009,1988,10.586250,2.359556
1940,10010,1988,15.140250,2.717357
2028,10011,1988,10.448438,2.346452
2175,10012,1988,4.765406,1.561383
3104,10016,1988,400.414000,5.992499


## 3.r_t+1

In [8]:
#月度收益率数据
RET=MonthlyStock[['PERMNO','year','month','RET']]
RET=RET.dropna()
RET=RET[RET['RET']!='C']
RET['RET']=RET['RET'].astype(str)
RET=RET[~RET['RET'].str.contains('B')]
RET['RET']=RET['RET'].astype(float)
RET.head(10)

,PERMNO,year,month,RET
2,10000,1986,2,-0.257143
3,10000,1986,3,0.365385
4,10000,1986,4,-0.098592
5,10000,1986,5,-0.222656
6,10000,1986,6,-0.005025
7,10000,1986,7,-0.080808
8,10000,1986,8,-0.615385
9,10000,1986,9,-0.057143
10,10000,1986,10,-0.242424
11,10000,1986,11,0.060000


In [26]:
#选取1989至2012年数据
RET_df=RET[(RET['year'] >= 1989) & (RET['year']<=2012)] 
RET_df['RET2']=RET_df['RET']+1
RET_df=RET_df.groupby(['year','PERMNO'])['RET2'].prod()-1
RET_df=RET_df.reset_index()
RET_df=RET_df.set_index("year")
RET_df.head(10)

,PERMNO,RET2
year,,
1989,10001,0.687926
1989,10002,-0.268657
1989,10003,-0.532995
1989,10005,-0.333334
1989,10007,2.220594
1989,10009,-0.322409
1989,10010,0.194444
1989,10011,-0.026666
1989,10012,-0.212120


In [27]:
#导入无风险利率
ff=pd.read_csv(open(path3))
ff=ff.set_index('year')
ff[['RF']]=ff[['RF']]/100
ff.head(10)

,Mkt-RF,SMB,HML,RF
year,,,,
1927,29.47,-2.46,-3.75,0.0312
1928,35.39,4.41,-5.83,0.0356
1929,-19.54,-30.78,11.96,0.0475
1930,-31.23,-5.19,-12.29,0.0241
1931,-45.11,3.51,-14.32,0.0107
1932,-9.39,4.91,10.49,0.0096
1933,57.05,48.86,28.15,0.0030
1934,3.02,25.43,-27.38,0.0016
1935,44.96,9.99,9.78,0.0017


In [28]:
#合并计算
RET_df=RET_df.merge(ff[['RF']],how='left',left_index=True,right_index=True)
RET_df['RET3']=RET_df['RET2']-RET_df['RF']
RET_df=RET_df[['PERMNO','RET3']].rename(columns={'RET3':'rt_1'})
RET_df=RET_df.reset_index()
RET_df['year']=RET_df['year']-1
RET_df.head(10)

,year,PERMNO,rt_1
0,1988,10001,0.604226
1,1988,10002,-0.352357
2,1988,10003,-0.616695
3,1988,10005,-0.417034
4,1988,10007,2.136894
5,1988,10009,-0.406109
6,1988,10010,0.110744
7,1988,10011,-0.110366
8,1988,10012,-0.295820
9,1988,10016,-0.191316


## 4.BM

In [29]:
#BM-----------------------------------------------------------------------
#查看基本面数据
columns4=getColumns(path4)
columns5=getColumns(path5)
#获取基本面数据
FundAnnu=pd.read_csv(open(path4),usecols=['GVKEY','LINKDT','LINKENDDT','datadate','seq','txdb','itcb','pstkrv','pstkl','pstk']) 
FundAnnu.head(10)

,GVKEY,LINKDT,LINKENDDT,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb
0,1000,19610101,19700929,19611231,0.0,NaN,0.0,NaN,NaN,0.000
1,1000,19610101,19700929,19621231,NaN,0.0,0.0,NaN,NaN,NaN
2,1000,19610101,19700929,19631231,0.0,0.0,0.0,0.0,0.553,0.008
3,1000,19610101,19700929,19641231,0.0,0.0,0.0,0.0,0.607,0.020
4,1000,19610101,19700929,19651231,0.0,0.0,0.0,0.0,0.491,0.000
5,1000,19610101,19700929,19661231,0.0,0.0,0.0,0.0,0.834,0.000
6,1000,19610101,19700929,19671231,0.0,0.0,0.0,0.0,0.744,0.000
7,1000,19610101,19700929,19681231,0.0,0.0,0.0,0.0,2.571,0.000
8,1000,19610101,19700929,19691231,0.0,0.0,0.0,0.0,10.211,0.000
9,1000,19701113,19780630,19701231,0.0,0.0,0.0,0.0,10.544,0.000


![title](fig/fig1.png)

In [30]:
#设置时间和类型转换
FundAnnu['year']=FundAnnu['datadate'].astype(str).str[:4]
name_list=['seq','txdb','itcb','pstkrv','pstkl','pstk']
for name in name_list:
    FundAnnu[name]=FundAnnu[name].astype(float)

In [31]:
#计算BE
def get_bvps(arr):
    if arr['pstkrv']!=0:
        return arr['pstkrv']
    elif arr['pstkl']!=0:
        return arr['pstkl']
    elif arr['pstk']!=0:
        return arr['pstk']
    else:
        return 0
FundAnnu['bvps']=FundAnnu.apply(get_bvps,axis=1)
FundAnnu['BE']=FundAnnu['seq']+FundAnnu['txdb']+FundAnnu['itcb']-FundAnnu['bvps']
FundAnnu.head(10)

,GVKEY,LINKDT,LINKENDDT,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,year,bvps,BE
0,1000,19610101,19700929,19611231,0.0,NaN,0.0,NaN,NaN,0.000,1961,NaN,NaN
1,1000,19610101,19700929,19621231,NaN,0.0,0.0,NaN,NaN,NaN,1962,NaN,NaN
2,1000,19610101,19700929,19631231,0.0,0.0,0.0,0.0,0.553,0.008,1963,0.0,0.561
3,1000,19610101,19700929,19641231,0.0,0.0,0.0,0.0,0.607,0.020,1964,0.0,0.627
4,1000,19610101,19700929,19651231,0.0,0.0,0.0,0.0,0.491,0.000,1965,0.0,0.491
5,1000,19610101,19700929,19661231,0.0,0.0,0.0,0.0,0.834,0.000,1966,0.0,0.834
6,1000,19610101,19700929,19671231,0.0,0.0,0.0,0.0,0.744,0.000,1967,0.0,0.744
7,1000,19610101,19700929,19681231,0.0,0.0,0.0,0.0,2.571,0.000,1968,0.0,2.571
8,1000,19610101,19700929,19691231,0.0,0.0,0.0,0.0,10.211,0.000,1969,0.0,10.211
9,1000,19701113,19780630,19701231,0.0,0.0,0.0,0.0,10.544,0.000,1970,0.0,10.544


In [32]:
#计算BM
FundAnnu2=MonthlyStock.copy()
FundAnnu2['MktCap']=np.abs(MonthlyStock['SHROUT']*MonthlyStock['ALTPRC'])/1000
#利用均值估算市值
FundAnnu2_sum=FundAnnu2.groupby(['year','PERMNO'])['MktCap'].mean().reset_index()

FundAnnu2_sum=FundAnnu2_sum.rename(columns = {'MktCap':'MktCap_adj'})
FundAnnu2=pd.merge(FundAnnu2,FundAnnu2_sum, how='inner', on=['year','PERMNO'])
FundAnnu2=FundAnnu2.sort_values(by = ['PERMNO','year']).drop_duplicates()
FundAnnu2=FundAnnu2[FundAnnu2['month'] == 12]
FundAnnu2.head(10)

,PERMNO,date,SHRCD,RET,SHROUT,ALTPRC,year,month,MktCap,MktCap_adj
11,10000,19861231,10.0,-0.377358,3843.0,-0.51563,1986,12,1.981566,9.175280
29,10001,19861231,11.0,0.015000,991.0,7.00000,1986,12,6.937000,6.339047
41,10001,19871231,11.0,-0.033535,992.0,5.87500,1987,12,5.828000,6.133870
53,10001,19881230,11.0,-0.021132,998.0,6.37500,1988,12,6.362250,6.393833
65,10001,19891229,11.0,0.037975,1022.0,10.12500,1989,12,10.347750,8.069958
77,10001,19901231,11.0,0.001299,1054.0,-9.50000,1990,12,10.013000,10.082552
89,10001,19911231,11.0,-0.006780,1075.0,14.50000,1991,12,15.587500,11.951969
101,10001,19921231,11.0,-0.015130,1080.0,14.00000,1992,12,15.120000,13.947240
113,10001,19931231,11.0,0.091176,1091.0,-18.37500,1993,12,20.047125,17.275625
126,10001,19941230,11.0,-0.033433,2224.0,8.00000,1994,12,17.792000,18.782144


In [34]:
#通过link匹配
c_crsp=pd.read_csv(open(path6),usecols=['gvkey','LPERMNO']) 
c_crsp=c_crsp.dropna()
c_crsp=c_crsp.drop_duplicates()
c_crsp.head()

,gvkey,LPERMNO
0,1000,25881.0
3,1001,10015.0
5,1002,10023.0
11,1003,10031.0
13,1004,54594.0


In [35]:
#匹配
BE_df=FundAnnu[['GVKEY','year','BE']]
BE_df=BE_df.merge(c_crsp,how='left',left_on='GVKEY',right_on='gvkey')
BE_df=BE_df.rename(columns={'LPERMNO':'PERMNO'})
BE_df=BE_df.sort_values('PERMNO')
BE_df=BE_df.dropna()
BE_df['year']=BE_df['year'].astype(int)

BM_df=BE_df.merge(FundAnnu2[['PERMNO','year','MktCap_adj']],how='left',on=['PERMNO','year'])
BM_df.head(10)

,GVKEY,year,BE,gvkey,PERMNO,MktCap_adj
0,13007,1985,-0.343,13007.0,10000.0,NaN
1,13007,1986,0.418,13007.0,10000.0,9.175280
2,12994,2005,23.768,12994.0,10001.0,24.192326
3,12994,2004,18.264,12994.0,10001.0,17.081384
4,12994,2003,21.113,12994.0,10001.0,17.733079
5,12994,2002,20.691,12994.0,10001.0,24.222683
6,12994,2001,19.846,12994.0,10001.0,27.865562
7,12994,2000,18.079,12994.0,10001.0,20.957635
8,12994,1999,17.537,12994.0,10001.0,21.272083
9,12994,1996,14.602,12994.0,10001.0,20.126859


In [36]:
#计算BM
BM_df=BM_df[BM_df['MktCap_adj']!=0]
BM_df['BM']=BM_df['BE']/BM_df['MktCap_adj']
BM_df=BM_df[['PERMNO','year','BM']]
BM_df=BM_df[(BM_df['year']>=1988) & (BM_df['year']<=2012)]
BM_df=BM_df.sort_values(['year','PERMNO'])
BM_df.head(10)

,PERMNO,year,BM
17,10001.0,1988,1.139535
32,10005.0,1988,1.250486
43,10006.0,1988,NaN
75,10007.0,1988,NaN
86,10010.0,1988,0.475011
100,10011.0,1988,0.016595
119,10012.0,1988,0.070117
145,10016.0,1988,1.315040
146,10016.0,1988,1.315040
194,10018.0,1988,0.811522


## 5.Beta

In [ ]:
#获取月度无风险收益率
ff_m=pd.read_csv(open(path7))
ff_m['year']=ff_m['time']//100
ff_m['month']=ff_m['time']%100
ff_m=ff_m.drop(['time','SMB','HML'],axis=1)

In [ ]:
#获取计算beta所需的收益率时间段
RET_df2=RET[(RET['year'] >= 1985) & (RET['year']<=2012)] 
RET_df2=RET_df2.merge(ff_m,how='left',on=['year','month'])
RET_df2['RET_adj']=RET_df2['RET']*100-RET_df2['RF']

In [ ]:
#全股票列表
permno_list=list(RET_df2['PERMNO'].unique())

In [ ]:
#计算beta
#输入permno，输出该股票beta时间序列
def beta(permno):
    df=RET_df2[RET_df2['PERMNO']==permno]
    year_list=list(df['year'].unique())
    if len(year_list)<=6:
        Res=pd.DataFrame({})
        return Res
    else:
        Res=pd.DataFrame({})
        for j in range(len(year_list)-5):
            year=year_list[j:j+5]
            dff=df[df['year'].isin(year)]
            dff=dff.rename(columns={'Mkt-RF':'Mkt_RF'})
            if len(dff)<24:
                beta=np.nan
            else:
                model=smf.ols('RET_adj ~ Mkt_RF',data=dff).fit()
                beta=model.params[1]
            res=pd.DataFrame({'PERMNO':permno,'year':year[-1],'beta':beta},index=[0])
            Res=pd.concat([Res,res])
        return Res

In [ ]:
#获取全市场beta
Beta_df=pd.DataFrame({})
for i in range(len(permno_list)):
    print(i)
    permno=permno_list[i]
    beta_df=beta(permno)
    Beta_df=pd.concat([Beta_df,beta_df])
Beta_df=Beta_df[(Beta_df['year']>=1988) & (Beta_df['year']<=2012)]
Beta_df=Beta_df.sort_values(['year','PERMNO'])

## 6.Alldata

In [45]:
path='D:/项目文件/项目文件-个人/Fintech/Fintech-Code/资产定价/alldata2021.csv'
alldata=pd.read_csv(open(path))
alldata.head(10)

,PERMNO,year,beta,BM,rt_1,MktCap,Size
0,10001,1988,0.072995,1.139535,0.604226,6.362250,1.850382
1,10002,1988,0.494006,NaN,-0.352357,9.840625,2.286519
2,10003,1988,0.797134,NaN,-0.616695,41.686000,3.730165
3,10005,1988,1.018545,1.250486,-0.417034,0.785250,-0.241753
4,10006,1988,NaN,NaN,NaN,NaN,NaN
5,10007,1988,NaN,NaN,2.136894,NaN,NaN
6,10009,1988,0.767990,NaN,-0.406109,10.586250,2.359556
7,10010,1988,1.287093,0.475011,0.110744,15.140250,2.717357
8,10011,1988,NaN,0.016595,-0.110366,10.448438,2.346452
9,10012,1988,0.734701,0.070117,-0.295820,4.765406,1.561383


## 7.SummaryStat

In [48]:
def summary(arr):
    mean=arr.mean()
    std=arr.std()
    
    skew=arr.skew()
    kurt=arr.kurt()
    
    Min=arr.min()
    q1=arr.quantile(0.05)
    q2=arr.quantile(0.25)
    Median=arr.quantile(0.5)
    q4=arr.quantile(0.75)
    q5=arr.quantile(0.95)
    Max=arr.max()
    
    n=arr.count()
    
    res=pd.DataFrame({'mean':mean,
                   'std':std,
                   'skew':skew,
                   'kurt':kurt,
                   'Min':Min,
                   'q1':q1,'q2':q2,'Median':Median,'q4':q4,'q5':q5,
                   'Max':Max,
                   'skew':skew,
                   'kurt':kurt,
                   'n':n},index=[0])
    return res

#全市场表格
resall=pd.DataFrame({})
for column in ['beta','MktCap','Size','BM','rt_1']:
    res=alldata.groupby('year')[column].apply(summary)
    res=res.reset_index()
    res=res.set_index('year')
    res=res.drop('level_1',axis=1)
    res=pd.DataFrame(res.mean()).T
    res.index=[column]
    resall=pd.concat([resall,res])
resall

,mean,std,skew,kurt,Min,q1,q2,Median,q4,q5,Max,n
beta,1.082040,0.965130,0.593144,22.070119,-6.546784,-0.149124,0.481590,0.955120,1.554737,2.775143,10.452792,5402.88
MktCap,2103.826680,10175.748510,13.869189,276.174587,0.359676,8.968958,48.733782,193.229202,842.123882,7960.807923,286996.495714,5966.16
Size,5.073902,2.081052,0.296286,-0.084874,-1.424307,1.888309,3.585190,4.937515,6.436030,8.750158,12.325271,5966.16
BM,0.707279,2.569939,-7.065964,981.772181,-93.181754,0.004143,0.276647,0.538746,0.910497,2.004547,40.731807,4871.56
rt_1,0.119618,0.789449,5.982598,129.199716,-1.019492,-0.665313,-0.263066,0.007010,0.311805,1.220029,19.181614,6087.20


In [49]:
#beta年度汇总
res1=alldata.groupby('year')['beta'].apply(summary)
res1=res1.reset_index()
res1=res1.set_index('year')
res1=res1.drop('level_1',axis=1)
res1

,mean,std,skew,kurt,Min,q1,q2,Median,q4,q5,Max,n
year,,,,,,,,,,,,
1988,1.017390,0.515681,0.489487,4.911529,-1.948805,0.232312,0.691735,1.014792,1.321029,1.834278,6.219887,5665
1989,1.021438,0.572205,-1.124516,30.135096,-10.294843,0.186386,0.681585,1.030799,1.350232,1.897578,7.104851,5858
1990,0.869566,0.968891,-7.372771,248.788551,-32.370270,-0.429557,0.410106,0.859719,1.351194,2.200345,8.384964,5650
1991,0.896763,0.820189,0.344589,3.660139,-4.152686,-0.293228,0.393683,0.864313,1.358167,2.234383,6.959574,5520
1992,0.903730,0.869196,0.063706,4.264793,-4.670071,-0.377032,0.376471,0.886122,1.388832,2.294004,8.204030,5512
1993,0.832305,1.188025,0.181173,13.514883,-10.657253,-0.889574,0.212417,0.793811,1.425864,2.678978,17.016006,5442
1994,0.968079,1.344183,4.260382,154.921468,-15.157924,-0.718801,0.334492,0.905019,1.572901,2.927036,41.792759,5774
1995,0.916563,1.106847,0.234522,5.211769,-7.275014,-0.675740,0.335994,0.848619,1.449478,2.707983,9.179647,6176
1996,0.969407,1.044817,0.751977,13.740609,-13.755407,-0.367232,0.371222,0.852284,1.443163,2.758608,11.599368,6413


In [50]:
res2=pd.DataFrame(res1.mean()).T
res2

,mean,std,skew,kurt,Min,q1,q2,Median,q4,q5,Max,n
0,1.08204,0.96513,0.593144,22.070119,-6.546784,-0.149124,0.48159,0.95512,1.554737,2.775143,10.452792,5402.88


## 缩尾和结尾

* 缩尾：简单地将一个大于或者小于某个确定值的变量设定为这个缺失值
* 截尾：简单地将被认定为极端值的变量设定为缺失。

## Newey和West调整

解决自相关和异方差的问题

当A表示一个是时间序列的变量，我们通常用T检验来验证A的均值是否等于某个特定值。但是某些情况下这个检验时不可靠的，比如当A具有自相关和异方差的问题，这会放大或者缩小其标准误差。

为了说明这一点，我们将A于一个常数进行回归，而不是用一个T检验。具体的说明方法见书P6

In [ ]:
def NWtest(a, lags = 5):
    # lags为滞后期
    adj_a = pd.DataFrame(a)
    adj_a = adj_a.dropna()
    if len(adj_a)>0:
        adj_a = adj_a.astype(float)
        adj_a = np.array(adj_a)
        model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type = 'HAC', cov_kwds={'maxlags': lags})
        return float(model.tvalues), float(model.pvalues)
    else:
        return [np.nan]*2